# 自然語言處理(NLP)應用

## 程式參考來源：
- https://keras.io/examples/nlp/text_classification_from_scratch/


## 影評(IMDB movie review)情緒分析 

### 資料集：https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
### 解壓縮後，刪除 train\unsup 目錄 

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

2024-10-26 14:58:05.699250: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-26 14:58:05.711828: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1729925885.724060   95164 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1729925885.727920   95164 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-26 14:58:05.741221: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [ ]:
# py3.12
# 注意:使用 NumPy 1.x , 目前(2024/10)還不適用NumPy 2.0

In [2]:
np.__version__
# 或: !pip show numpy

'2.0.2'

## 建立訓練及驗證資料集

In [3]:
# 在PC執行時, 需將整個資料夾(程式碼與acImdb夾)放在使用者/名稱下
batch_size = 32
raw_train_ds = tf.keras.preprocessing.text_dataset_from_directory(
    "./data/aclImdb/train",
    batch_size=batch_size,
    validation_split=0.2,
    subset="training",
    seed=1337,
)
raw_val_ds = tf.keras.preprocessing.text_dataset_from_directory(
    "./data/aclImdb/train",
    batch_size=batch_size,
    validation_split=0.2,
    subset="validation",
    seed=1337,
)
raw_test_ds = tf.keras.preprocessing.text_dataset_from_directory(
    "./data/aclImdb/test", batch_size=batch_size
)

print(
    "Number of batches in raw_train_ds: %d"
    % tf.data.experimental.cardinality(raw_train_ds)
)
print(
    "Number of batches in raw_val_ds: %d" % tf.data.experimental.cardinality(raw_val_ds)
)
print(
    "Number of batches in raw_test_ds: %d"
    % tf.data.experimental.cardinality(raw_test_ds)
)

Found 25000 files belonging to 2 classes.
Using 20000 files for training.


I0000 00:00:1729925892.765419   95164 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5563 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9


Found 25000 files belonging to 2 classes.
Using 5000 files for validation.
Found 25000 files belonging to 2 classes.
Number of batches in raw_train_ds: 625
Number of batches in raw_val_ds: 157
Number of batches in raw_test_ds: 782


## 查看5筆資料內容

In [4]:
for text_batch, label_batch in raw_train_ds.take(1):
    for i in range(5):
        print(text_batch.numpy()[i])
        print(label_batch.numpy()[i])

b'I\'ve seen tons of science fiction from the 70s; some horrendously bad, and others thought provoking and truly frightening. Soylent Green fits into the latter category. Yes, at times it\'s a little campy, and yes, the furniture is good for a giggle or two, but some of the film seems awfully prescient. Here we have a film, 9 years before Blade Runner, that dares to imagine the future as somthing dark, scary, and nihilistic. Both Charlton Heston and Edward G. Robinson fare far better in this than The Ten Commandments, and Robinson\'s assisted-suicide scene is creepily prescient of Kevorkian and his ilk. Some of the attitudes are dated (can you imagine a filmmaker getting away with the "women as furniture" concept in our oh-so-politically-correct-90s?), but it\'s rare to find a film from the Me Decade that actually can make you think. This is one I\'d love to see on the big screen, because even in a widescreen presentation, I don\'t think the overall scope of this film would receive its

2024-10-26 14:58:19.158896: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


## 前置處理，刪除跳行標籤(br)

In [5]:
# from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras.layers import TextVectorization

import string
import re


# 刪除跳行標籤(<br />)
def custom_standardization(input_data):
    lowercase = tf.strings.lower(input_data)
    stripped_html = tf.strings.regex_replace(lowercase, "<br />", " ")
    return tf.strings.regex_replace(
        stripped_html, "[%s]" % re.escape(string.punctuation), ""
    )


In [6]:
# 參數設定
max_features = 20000
embedding_dim = 128
sequence_length = 500

# TextVectorization 參考 https://www.tensorflow.org/api_docs/python/tf/keras/layers/experimental/preprocessing/TextVectorization
# 1. 轉成小寫，去除標點符號 (lowercasing + punctuation stripping)
# 2. 分詞 (split each sample into words)
# 3. 重組為 ngrams
# 4. 將單字對應成整數
# 5. 把樣本轉換為向量
vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=max_features,
    output_mode="int",
    output_sequence_length=sequence_length,
)

# 去除 labels 欄位
text_ds = raw_train_ds.map(lambda x, y: x)
# 呼叫 adapt 函數建立字彙表
vectorize_layer.adapt(text_ds)

2024-10-26 14:58:29.603997: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [7]:
type(text_ds), text_ds

(tensorflow.python.data.ops.map_op._MapDataset,
 <_MapDataset element_spec=TensorSpec(shape=(None,), dtype=tf.string, name=None)>)

## 呼叫 vectorize_layer 轉換資料

In [8]:
# vectorize_layer 有兩種用法
# 1. 放進 model 內
"""
text_input = tf.keras.Input(shape=(1,), dtype=tf.string, name='text')
x = vectorize_layer(text_input)
x = layers.Embedding(max_features + 1, embedding_dim)(x)
"""
# 2. 直接對 dataset 作轉換： 可非同步處理或使用GPU，獲得較佳的效能，以下採此作法。


def vectorize_text(text, label):
    text = tf.expand_dims(text, -1)
    return vectorize_layer(text), label


# 呼叫 vectorize_layer 轉換資料
train_ds = raw_train_ds.map(vectorize_text)
val_ds = raw_val_ds.map(vectorize_text)
test_ds = raw_test_ds.map(vectorize_text)

# 呼叫 cache/prefetch，獲得較佳的效能
train_ds = train_ds.cache().prefetch(buffer_size=10)
val_ds = val_ds.cache().prefetch(buffer_size=10)
test_ds = test_ds.cache().prefetch(buffer_size=10)

## 建立卷積模型(Conv1D)

In [9]:
from tensorflow.keras import layers

# A integer input for vocab indices.
inputs = tf.keras.Input(shape=(None,), dtype="int64")

# Next, we add a layer to map those vocab indices into a space of dimensionality
# 'embedding_dim'.
x = layers.Embedding(max_features, embedding_dim)(inputs)
x = layers.Dropout(0.5)(x)

# Conv1D + global max pooling
x = layers.Conv1D(128, 7, padding="valid", activation="relu", strides=3)(x)
x = layers.Conv1D(128, 7, padding="valid", activation="relu", strides=3)(x)
x = layers.GlobalMaxPooling1D()(x)

# We add a vanilla hidden layer:
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.5)(x)

# We project onto a single unit output layer, and squash it with a sigmoid:
predictions = layers.Dense(1, activation="sigmoid", name="predictions")(x)

model = tf.keras.Model(inputs, predictions)

# Compile the model with binary crossentropy loss and an adam optimizer.
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

## 訓練模型

In [10]:
epochs = 3
model.fit(train_ds, validation_data=val_ds, epochs=epochs)

Epoch 1/3


I0000 00:00:1729925923.066508   95272 service.cc:148] XLA service 0x7f25a000b250 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1729925923.066556   95272 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce RTX 4060 Laptop GPU, Compute Capability 8.9
2024-10-26 14:58:43.103412: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1729925923.270269   95272 cuda_dnn.cc:529] Loaded cuDNN version 90300


 14/625 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.5087 - loss: 0.6943

I0000 00:00:1729925925.074305   95272 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


625/625 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - accuracy: 0.5837 - loss: 0.6273 - val_accuracy: 0.8710 - val_loss: 0.3175
Epoch 2/3
625/625 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.8852 - loss: 0.2799 - val_accuracy: 0.8776 - val_loss: 0.3174
Epoch 3/3
625/625 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9476 - loss: 0.1470 - val_accuracy: 0.8568 - val_loss: 0.5138


## 模型評估

In [11]:
print(model.evaluate(test_ds))

782/782 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.8524 - loss: 0.5275
[0.5224646329879761, 0.8526399731636047]


## 新資料預測

In [12]:
# A string input
inputs = tf.keras.Input(shape=(1,), dtype="string")
# Turn strings into vocab indices
indices = vectorize_layer(inputs)
# Turn vocab indices into predictions
outputs = model(indices)

# 建立一個模型，使用已訓練的權重(in outputs)
end_to_end_model = tf.keras.Model(inputs, outputs)
end_to_end_model.compile(
    loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"]
)

# Test it with `raw_test_ds`, which yields raw strings
end_to_end_model.evaluate(raw_test_ds)

782/782 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.8522 - loss: 0.5294


[0.5224645137786865, 0.8526399731636047]

## Bidirectional LSTM on IMDB

## 建立LSTM模型

In [13]:
# Input for variable-length sequences of integers
inputs = keras.Input(shape=(None,), dtype="int32")
# Embed each integer in a 128-dimensional vector
x = layers.Embedding(max_features, 128)(inputs)
# Add 2 bidirectional LSTMs
x = layers.Bidirectional(layers.LSTM(64, return_sequences=True))(x)
x = layers.Bidirectional(layers.LSTM(64))(x)
# Add a classifier
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, None)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_1 (Embedding)         │ (None, None, 128)      │     2,560,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, None, 128)      │        98,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ (None, 128)            │        98,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,757,761 (10.52 MB)

 Trainable params: 2,757,761 (10.52 MB)

 Non-trainable params: 0 (0.00 B)

In [14]:
model.compile("adam", "binary_crossentropy", metrics=["accuracy"])
model.fit(train_ds, batch_size=32, epochs=3, validation_data=val_ds)

Epoch 1/3
625/625 ━━━━━━━━━━━━━━━━━━━━ 100s 155ms/step - accuracy: 0.7005 - loss: 0.5605 - val_accuracy: 0.8498 - val_loss: 0.3852
Epoch 2/3
625/625 ━━━━━━━━━━━━━━━━━━━━ 94s 151ms/step - accuracy: 0.8109 - loss: 0.4184 - val_accuracy: 0.8652 - val_loss: 0.3409
Epoch 3/3
625/625 ━━━━━━━━━━━━━━━━━━━━ 92s 147ms/step - accuracy: 0.9079 - loss: 0.2440 - val_accuracy: 0.8516 - val_loss: 0.3649


## 模型評估

In [14]:
print(model.evaluate(test_ds))

782/782 ━━━━━━━━━━━━━━━━━━━━ 49s 63ms/step - accuracy: 0.4916 - loss: 246.2453
[242.1832275390625, 0.5]


## 新資料預測

In [15]:
# A string input
inputs = tf.keras.Input(shape=(1,), dtype="string")
# Turn strings into vocab indices
indices = vectorize_layer(inputs)
# Turn vocab indices into predictions
outputs = model(indices)

# 建立一個模型，使用已訓練的權重(in outputs)
end_to_end_model = tf.keras.Model(inputs, outputs)
end_to_end_model.compile(
    loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"]
)

# Test it with `raw_test_ds`, which yields raw strings
end_to_end_model.evaluate(raw_test_ds)

782/782 ━━━━━━━━━━━━━━━━━━━━ 53s 67ms/step - accuracy: 0.4934 - loss: 245.4008


[242.18295288085938, 0.5]